In [1]:
import tensorflow as tf
import numpy as np

from text_processing import quixote_data_cleaning, text_processing, quixote_traintest_split, validation_text_processing
from RNN_class import Basic_RNN, OneStep, Basic_RNN_v2

The aim of this notebook is to demonsrate how the data is imported, cleaned, processed and used to train and evaluate a simple model.

In [2]:
#run the data importing and cleaning function
text=quixote_data_cleaning()
train_text,validation_text= quixote_traintest_split(text)

sequence_length=64
BATCH_SIZE = 64
BUFFER_SIZE = 10000

#run the data processing function, takes a string and returns a tf.data object for efficent implementation in an RNN
dataset,ids_from_chars, chars_from_ids, vocab_size=text_processing(train_text,sequence_length,BATCH_SIZE,BUFFER_SIZE)

#produce the validation text (last chapter of the novel) as well.
valid_dataset=validation_text_processing(validation_text,sequence_length,BATCH_SIZE,BUFFER_SIZE,ids_from_chars)

## Training and Testing a basic model.

In [5]:
embedding_dim=256
hidden_units=128

basic_model = Basic_RNN(hidden_units, vocab_size,embedding_dim,learning_rate=2e-3)
EPOCHS=10
basic_model.fit(dataset,epochs=EPOCHS,validation_data=valid_dataset, callbacks=[basic_model.early_stopping])

Epoch 1/10
 21/520 ━━━━━━━━━━━━━━━━━━━━ 22s 46ms/step - accuracy: 0.1636 - loss: 3.6227

In [ ]:
#test the output of the basic RNN model.
temperature=0.6
one_step_model = OneStep(basic_model, chars_from_ids, ids_from_chars, temperature)
text='Don Quixote '

result=one_step_model.generate_more_steps(10,text)
print("Basic Model: {}".format(result))

Basic Model: [b'Don Quixote o hind s b']


This is a very bad model but that is to be expected as it only took five minutes to train. 
It seems to understand how spaces and commas work but cannot produce words.

## Testing the effects of dropout
Models without dropout are faster to train (as there are fewer parameters), however their loss scores will be higher.
This may not be a problem as decreased accuracy on the training data has the advantage of being more robust.
An issue with some models is that once an unusual character is chosen it de-rails the following characters, this property is not measured in any of the metrics.

In [21]:
# embedding_dim=256
# hidden_units=512

# #training a standard RNN model with 3 RNN layers and a dropout rate of 0.5 and recurrent dropout rate of 0.5.
# v2_model = Basic_RNN_v2(hidden_units, vocab_size,embedding_dim,dropout_rate=0.5,recurrent_dropout_rate=0.5,learning_rate=2e-3)
# EPOCHS=20
# v2_model.fit(dataset,epochs=EPOCHS)

# #same model as above but with no dropout.
# v2_model_nodropout = Basic_RNN_v2(hidden_units, vocab_size,embedding_dim,dropout_rate=0.0,recurrent_dropout_rate=0.0,learning_rate=2e-3)
# EPOCHS=20
# v2_model_nodropout.fit(dataset,epochs=EPOCHS)

# #save the models
# v2_model.save('saved_models/RNN_14062024.keras')
# v2_model_nodropout.save('saved_models/RNN_nodropout_14062024.keras')

In [3]:
#load models
v2_model = tf.keras.models.load_model('saved_models/RNN_14062024.keras')
v2_model_nodropout= tf.keras.models.load_model('saved_models/RNN_nodropout_14062024.keras')

In [12]:
temperature=0.6
text='Don Quixote '

#compare models with a simple (before seen input)
v2_one_step_model = OneStep(v2_model, chars_from_ids, ids_from_chars, temperature)
v2_result=v2_one_step_model.generate_more_steps(20,text)
print(v2_result)

v2_nodropout_one_step_model = OneStep(v2_model, chars_from_ids, ids_from_chars, temperature)
v2_nodropout_result=v2_nodropout_one_step_model.generate_more_steps(20,text)
print(v2_nodropout_result)

tf.Tensor([b'Don Quixote w STof mashe hy ofin'], shape=(1,), dtype=string)
tf.Tensor([b'Don Quixote e  w m w ms w torim '], shape=(1,), dtype=string)


In [14]:
#compare models using the start of the test data
text="CHAPTER LXXIV. OF HOW DON QUIXOTE FELL SICK, AND OF THE WILL HE MADE, AND HOW HE DIED"

v2_result=v2_one_step_model.generate_more_steps(40,text)
print("Dropout model: {}".format(v2_result))

v2_nodropout_result=v2_nodropout_one_step_model.generate_more_steps(40,text)
print("No Dropout model: {})".format(v2_nodropout_result))

Dropout model: [b'CHAPTER LXXIV. OF HOW DON QUIXOTE FELL SICK, AND OF THE WILL HE MADE, AND HOW HE DIEDL he aldind thend mit m hes w pld t SIw ']
No Dropout model: [b'CHAPTER LXXIV. OF HOW DON QUIXOTE FELL SICK, AND OF THE WILL HE MADE, AND HOW HE DIEDH w whe and wof w o s angk s wos w he w '])
